In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
import pandas as pd
import os
import random

os.environ["CUDA_VISIBLE_DEVICES"]="-1"

# 2nd eLU, tanh, 4, 50, 50, 50, 1
## Model
## Train NN & Predict

Import Data (normalized) & Separate to Training and Test

In [2]:
#data = np.loadtxt("ACL nomalization data.txt")
#data2 = np.loadtxt("acl data.txt")
data = np.loadtxt("ACL FTIR data_mean.txt")

X_NormalizedData = data[:,0:4]
X_NormalizedData = X_NormalizedData - 0.5
Y_NormalizedData = data[:,4]

X_TrainingData = data[0:20,0:4]
X_TrainingData = X_TrainingData - 0.5
Y_TrainingData = data[0:20,4]

X_TestData = data[20:,0:4]
X_TestData = X_TestData - 0.5
Y_TestData = data[20:,4]

Sequence = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]

#XY_TrainingData = data[0:21,:]
#X_TrainingData = XY_TrainingData[0:21,0:4]
#Y_TrainingData = XY_TrainingData[0:21,4]
#
#XY_TestData = data[21:,:]
#X_TestData = XY_TestData[:,0:4]
#Y_TestData = XY_TestData[:,4]

print(X_TrainingData)
print("------------------------")
print(X_TestData)
print("------------------------")
print(Y_TrainingData)
print("------------------------")
print(Y_TestData)

[[ 0.   0.   0.5  0.5]
 [ 0.  -0.5  0.   0.5]
 [-0.5 -0.5  0.   0. ]
 [-0.5  0.   0.5  0. ]
 [ 0.5  0.   0.   0.5]
 [ 0.   0.  -0.5  0.5]
 [ 0.5 -0.5  0.   0. ]
 [ 0.   0.  -0.5 -0.5]
 [ 0.  -0.5 -0.5  0. ]
 [ 0.   0.   0.5 -0.5]
 [-0.5  0.   0.  -0.5]
 [ 0.5  0.5  0.   0. ]
 [ 0.  -0.5  0.5  0. ]
 [-0.5  0.5  0.   0. ]
 [ 0.5  0.   0.  -0.5]
 [-0.5  0.  -0.5  0. ]
 [ 0.   0.5  0.5  0. ]
 [ 0.   0.5  0.  -0.5]
 [ 0.5  0.   0.5  0. ]
 [ 0.   0.5 -0.5  0. ]]
------------------------
[[ 0.  -0.5  0.  -0.5]
 [ 0.5  0.  -0.5  0. ]
 [-0.5  0.   0.   0.5]
 [ 0.   0.   0.   0. ]
 [ 0.   0.5  0.   0.5]]
------------------------
[-0.21844449 -0.47152439 -0.45735005 -0.1123416  -0.11245536 -0.18036006
 -0.19692817  0.00145492 -0.14520267  0.01178692  0.04523337  0.52847561
 -0.13421128  0.19149351 -0.00787756  0.09143864 -0.16016158  0.30505231
  0.04222738  0.33060721]
------------------------
[-0.08104142  0.20294599  0.18485121  0.07951933  0.2628122 ]


# 1st eLU, tanh, 4, 18, 18, 9, 9, 1
R_square of EntireData
0.457788879515
R_square of TestData
-8.85773593135

## Model

In [3]:
learning_rate = 0.001
tf.set_random_seed(777)

n_InputUnit = 4
n_1stHiddenUnit=18
n_2ndHiddenUnit=18
n_3rdHiddenUnit=9
n_4thHiddenUnit=9
n_OutputUnit = 1

W1 = tf.Variable(tf.random_normal([n_InputUnit, n_1stHiddenUnit],stddev=0.01))
W2 = tf.Variable(tf.random_normal([n_1stHiddenUnit, n_2ndHiddenUnit],stddev=0.01))
W3 = tf.Variable(tf.random_normal([n_2ndHiddenUnit, n_3rdHiddenUnit],stddev=0.01))
W4 = tf.Variable(tf.random_normal([n_3rdHiddenUnit, n_4thHiddenUnit],stddev=0.01))
W5 = tf.Variable(tf.random_normal([n_4thHiddenUnit, n_OutputUnit],stddev=0.01))

b1 = tf.Variable(tf.random_normal([n_1stHiddenUnit], stddev=0.01))
b2 = tf.Variable(tf.random_normal([n_2ndHiddenUnit], stddev=0.01))
b3 = tf.Variable(tf.random_normal([n_3rdHiddenUnit], stddev=0.01))
b4 = tf.Variable(tf.random_normal([n_4thHiddenUnit], stddev=0.01))
b5 = tf.Variable(tf.random_normal([n_OutputUnit], stddev=0.01))

X = tf.placeholder(tf.float32, [None, 4]) # tensorflow의 placeholder라는 변수를 선언
Y = tf.placeholder(tf.float32, [None,])

L1 = tf.nn.elu(tf.matmul(X, W1) + b1)
L2 = tf.nn.elu(tf.matmul(L1, W2) + b2)
L3 = tf.nn.elu(tf.matmul(L2, W3) + b3)
L4 = tf.nn.elu(tf.matmul(L3, W4) + b4)
Hypothesis = tf.nn.tanh(tf.matmul(L4, W5) + b5)

cost = tf.reduce_mean(tf.square(Y-Hypothesis))
train = tf.train.AdamOptimizer(learning_rate=learning_rate,beta1=0.5).minimize(cost)

## Train NN & Predict

In [4]:
with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    
    for step in range(100000):
        SGDorder = Sequence[step % 20] # training을 0번부터 20번까지 순차적으로 돌린다.
        session.run(train, feed_dict ={X: X_TrainingData[SGDorder: SGDorder + 1,:], Y: Y_TrainingData[SGDorder: SGDorder + 1]})
        #session.run(train, feed_dict ={X: X_TrainingData, Y: Y_TrainingData})
        
        if step % 1000 == 0:
            print (SGDorder,step, session.run(cost, feed_dict = {X: X_TrainingData, Y: Y_TrainingData}))
            #print (W1)
            #print (b1)
        if step % 20 == 0:
            random.shuffle(Sequence)
            
    Predict_to_TestData = session.run(Hypothesis, feed_dict={X : X_TestData})
    Predict_to_TrainingData = session.run(Hypothesis, feed_dict={X : X_TrainingData})
    Predict_to_EntireData = session.run(Hypothesis, feed_dict = {X : X_NormalizedData})
    
    print("R_square of EntireData")
    print(r2_score(Y_NormalizedData, Predict_to_EntireData))
    print("R_square of TestData")
    print(r2_score(Y_TestData, Predict_to_TestData))
    print("Result of predicting TrainingData")
    print(Predict_to_TrainingData)
    print("Result of predicting TestData")
    print(Predict_to_TestData)
    print("Y_NormalizedData")
    print(Y_NormalizedData)

0 0 0.0591896
10 1000 0.0980585
11 2000 0.0964576
5 3000 0.0933272
4 4000 0.0976842
3 5000 0.101685
17 6000 0.100689
9 7000 0.093919
3 8000 0.104772
19 9000 0.105406
7 10000 0.104006
8 11000 0.107245
19 12000 0.105855
3 13000 0.107375
0 14000 0.108926
15 15000 0.102045
12 16000 0.11168
15 17000 0.102177
1 18000 0.108222
0 19000 0.105598
0 20000 0.108124
3 21000 0.111011
14 22000 0.111952
2 23000 0.108096
2 24000 0.106436
14 25000 0.107102
16 26000 0.110736
10 27000 0.106293
3 28000 0.106177
1 29000 0.113394
7 30000 0.109856
0 31000 0.108528
11 32000 0.112121
0 33000 0.10653
15 34000 0.111205
0 35000 0.11291
8 36000 0.104547
3 37000 0.107129
17 38000 0.114612
2 39000 0.112383
11 40000 0.104235
3 41000 0.114119
8 42000 0.115389
14 43000 0.112666
13 44000 0.112508
13 45000 0.115128
19 46000 0.115
18 47000 0.114319
1 48000 0.114413
13 49000 0.110813
19 50000 0.110752
3 51000 0.113449
14 52000 0.115576
1 53000 0.113428
9 54000 0.11659
18 55000 0.117161
11 56000 0.115485
7 57000 0.110828
11 

# 2nd eLU, tanh, 4, 50, 50, 50, 50, 1
R_square of EntireData
0.643568734249
R_square of TestData
-5.43605667929

## Model

In [5]:
learning_rate = 0.001
tf.set_random_seed(777)

n_InputUnit = 4
n_1stHiddenUnit=50
n_2ndHiddenUnit=50
n_3rdHiddenUnit=50
n_4thHiddenUnit=50
n_OutputUnit = 1

W1 = tf.Variable(tf.random_normal([n_InputUnit, n_1stHiddenUnit],stddev=0.01))
W2 = tf.Variable(tf.random_normal([n_1stHiddenUnit, n_2ndHiddenUnit],stddev=0.01))
W3 = tf.Variable(tf.random_normal([n_2ndHiddenUnit, n_3rdHiddenUnit],stddev=0.01))
W4 = tf.Variable(tf.random_normal([n_3rdHiddenUnit, n_4thHiddenUnit],stddev=0.01))
W5 = tf.Variable(tf.random_normal([n_4thHiddenUnit, n_OutputUnit],stddev=0.01))

b1 = tf.Variable(tf.random_normal([n_1stHiddenUnit], stddev=0.01))
b2 = tf.Variable(tf.random_normal([n_2ndHiddenUnit], stddev=0.01))
b3 = tf.Variable(tf.random_normal([n_3rdHiddenUnit], stddev=0.01))
b4 = tf.Variable(tf.random_normal([n_4thHiddenUnit], stddev=0.01))
b5 = tf.Variable(tf.random_normal([n_OutputUnit], stddev=0.01))

X = tf.placeholder(tf.float32, [None, 4]) # tensorflow의 placeholder라는 변수를 선언
Y = tf.placeholder(tf.float32, [None,])

L1 = tf.nn.elu(tf.matmul(X, W1) + b1)
L2 = tf.nn.elu(tf.matmul(L1, W2) + b2)
L3 = tf.nn.elu(tf.matmul(L2, W3) + b3)
L4 = tf.nn.elu(tf.matmul(L3, W4) + b4)
Hypothesis = tf.nn.tanh(tf.matmul(L4, W5) + b5)

cost = tf.reduce_mean(tf.square(Y-Hypothesis))
train = tf.train.AdamOptimizer(learning_rate=learning_rate,beta1=0.5).minimize(cost)

## Train NN & Predict

In [6]:
with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    
    for step in range(100000):
        SGDorder = Sequence[step % 20] # training을 0번부터 20번까지 순차적으로 돌린다.
        session.run(train, feed_dict ={X: X_TrainingData[SGDorder: SGDorder + 1,:], Y: Y_TrainingData[SGDorder: SGDorder + 1]})
        #session.run(train, feed_dict ={X: X_TrainingData, Y: Y_TrainingData})
        
        if step % 1000 == 0:
            print (SGDorder,step, session.run(cost, feed_dict = {X: X_TrainingData, Y: Y_TrainingData}))
            #print (W1)
            #print (b1)
        if step % 20 == 0:
            random.shuffle(Sequence)
            
    Predict_to_TestData = session.run(Hypothesis, feed_dict={X : X_TestData})
    Predict_to_TrainingData = session.run(Hypothesis, feed_dict={X : X_TrainingData})
    Predict_to_EntireData = session.run(Hypothesis, feed_dict = {X : X_NormalizedData})
    
    print("R_square of EntireData")
    print(r2_score(Y_NormalizedData, Predict_to_EntireData))
    print("R_square of TestData")
    print(r2_score(Y_TestData, Predict_to_TestData))
    print("Result of predicting TrainingData")
    print(Predict_to_TrainingData)
    print("Result of predicting TestData")
    print(Predict_to_TestData)
    print("Y_NormalizedData")
    print(Y_NormalizedData)

16 0 0.0595077
11 1000 0.09946
1 2000 0.0877738
17 3000 0.0936581
4 4000 0.0909709
6 5000 0.0900273
18 6000 0.0945143
7 7000 0.101718
0 8000 0.101703
11 9000 0.123767
2 10000 0.109589
18 11000 0.103871
19 12000 0.112006
1 13000 0.106627
6 14000 0.114356
9 15000 0.117015
17 16000 0.116913
12 17000 0.110534
0 18000 0.124202
16 19000 0.109804
18 20000 0.0977262
8 21000 0.108214
4 22000 0.115356
12 23000 0.113756
9 24000 0.112429
10 25000 0.112796
8 26000 0.113025
1 27000 0.115756
6 28000 0.11383
6 29000 0.114028
5 30000 0.115787
6 31000 0.125865
1 32000 0.127024
19 33000 0.117203
14 34000 0.114264
17 35000 0.118072
14 36000 0.11633
18 37000 0.134329
12 38000 0.126759
8 39000 0.125134
10 40000 0.112289
19 41000 0.112127
7 42000 0.113849
7 43000 0.135997
4 44000 0.11409
7 45000 0.123144
10 46000 0.111609
17 47000 0.11433
15 48000 0.115294
0 49000 0.116343
12 50000 0.104387
11 51000 0.115065
6 52000 0.115178
9 53000 0.116986
3 54000 0.116174
2 55000 0.114511
6 56000 0.11295
7 57000 0.114964


# 3rd eLU, tanh, 4, 10, 10, 10, 1
R_square of EntireData
0.360907984837
R_square of TestData
-10.5949482088

## Model

In [7]:
learning_rate = 0.001
tf.set_random_seed(777)

n_InputUnit = 4
n_1stHiddenUnit=10
n_2ndHiddenUnit=10
n_3rdHiddenUnit=10
n_OutputUnit = 1

W1 = tf.Variable(tf.random_normal([n_InputUnit, n_1stHiddenUnit],stddev=0.01))
W2 = tf.Variable(tf.random_normal([n_1stHiddenUnit, n_2ndHiddenUnit],stddev=0.01))
W3 = tf.Variable(tf.random_normal([n_2ndHiddenUnit, n_3rdHiddenUnit],stddev=0.01))
W4 = tf.Variable(tf.random_normal([n_3rdHiddenUnit, n_OutputUnit],stddev=0.01))

b1 = tf.Variable(tf.random_normal([n_1stHiddenUnit], stddev=0.01))
b2 = tf.Variable(tf.random_normal([n_2ndHiddenUnit], stddev=0.01))
b3 = tf.Variable(tf.random_normal([n_3rdHiddenUnit], stddev=0.01))
b4 = tf.Variable(tf.random_normal([n_OutputUnit], stddev=0.01))

X = tf.placeholder(tf.float32, [None, 4]) # tensorflow의 placeholder라는 변수를 선언
Y = tf.placeholder(tf.float32, [None,])

L1 = tf.nn.elu(tf.matmul(X, W1) + b1)
L2 = tf.nn.elu(tf.matmul(L1, W2) + b2)
L3 = tf.nn.elu(tf.matmul(L2, W3) + b3)
Hypothesis = tf.nn.tanh(tf.matmul(L3, W4) + b4)

cost = tf.reduce_mean(tf.square(Y-Hypothesis))
train = tf.train.AdamOptimizer(learning_rate=learning_rate,beta1=0.5).minimize(cost)

## Train NN & Predict

In [8]:
with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    
    for step in range(100000):
        SGDorder = Sequence[step % 20] # training을 0번부터 20번까지 순차적으로 돌린다.
        session.run(train, feed_dict ={X: X_TrainingData[SGDorder: SGDorder + 1,:], Y: Y_TrainingData[SGDorder: SGDorder + 1]})
        #session.run(train, feed_dict ={X: X_TrainingData, Y: Y_TrainingData})
        
        if step % 1000 == 0:
            print (SGDorder,step, session.run(cost, feed_dict = {X: X_TrainingData, Y: Y_TrainingData}))
            #print (W1)
            #print (b1)
        if step % 20 == 0:
            random.shuffle(Sequence)
            
    Predict_to_TestData = session.run(Hypothesis, feed_dict={X : X_TestData})
    Predict_to_TrainingData = session.run(Hypothesis, feed_dict={X : X_TrainingData})
    Predict_to_EntireData = session.run(Hypothesis, feed_dict = {X : X_NormalizedData})
    
    print("R_square of EntireData")
    print(r2_score(Y_NormalizedData, Predict_to_EntireData))
    print("R_square of TestData")
    print(r2_score(Y_TestData, Predict_to_TestData))
    print("Result of predicting TrainingData")
    print(Predict_to_TrainingData)
    print("Result of predicting TestData")
    print(Predict_to_TestData)
    print("Y_NormalizedData")
    print(Y_NormalizedData)

16 0 0.058422
16 1000 0.0997606
1 2000 0.094608
8 3000 0.0967422
15 4000 0.0963633
13 5000 0.0987266
1 6000 0.0961981
18 7000 0.0971074
18 8000 0.0974882
3 9000 0.0993656
18 10000 0.101455
11 11000 0.103075
15 12000 0.102778
10 13000 0.103675
5 14000 0.10346
10 15000 0.104664
10 16000 0.105998
7 17000 0.103855
8 18000 0.105087
6 19000 0.105836
17 20000 0.10622
15 21000 0.107037
14 22000 0.106481
14 23000 0.105749
11 24000 0.10486
4 25000 0.107782
7 26000 0.10366
17 27000 0.105192
18 28000 0.107147
5 29000 0.10502
14 30000 0.105536
9 31000 0.104026
3 32000 0.104841
3 33000 0.106717
5 34000 0.110441
13 35000 0.106308
10 36000 0.108114
8 37000 0.108384
8 38000 0.101711
5 39000 0.105369
15 40000 0.107707
17 41000 0.113406
5 42000 0.11125
13 43000 0.106797
7 44000 0.106032
19 45000 0.113299
7 46000 0.111743
19 47000 0.110955
10 48000 0.114773
16 49000 0.112712
18 50000 0.107372
16 51000 0.112951
1 52000 0.115978
9 53000 0.110534
8 54000 0.111508
8 55000 0.112613
15 56000 0.115948
12 57000 0

# 4th eLU, tanh, 4, 50, 30, 10, 1
R_square of EntireData
0.672840700519
R_square of TestData
-4.92750092321

## Model

In [9]:
learning_rate = 0.001
tf.set_random_seed(777)

n_InputUnit = 4
n_1stHiddenUnit=50
n_2ndHiddenUnit=30
n_3rdHiddenUnit=10
n_OutputUnit = 1

W1 = tf.Variable(tf.random_normal([n_InputUnit, n_1stHiddenUnit],stddev=0.01))
W2 = tf.Variable(tf.random_normal([n_1stHiddenUnit, n_2ndHiddenUnit],stddev=0.01))
W3 = tf.Variable(tf.random_normal([n_2ndHiddenUnit, n_3rdHiddenUnit],stddev=0.01))
W4 = tf.Variable(tf.random_normal([n_3rdHiddenUnit, n_OutputUnit],stddev=0.01))

b1 = tf.Variable(tf.random_normal([n_1stHiddenUnit], stddev=0.01))
b2 = tf.Variable(tf.random_normal([n_2ndHiddenUnit], stddev=0.01))
b3 = tf.Variable(tf.random_normal([n_3rdHiddenUnit], stddev=0.01))
b4 = tf.Variable(tf.random_normal([n_OutputUnit], stddev=0.01))

X = tf.placeholder(tf.float32, [None, 4]) # tensorflow의 placeholder라는 변수를 선언
Y = tf.placeholder(tf.float32, [None,])

L1 = tf.nn.elu(tf.matmul(X, W1) + b1)
L2 = tf.nn.elu(tf.matmul(L1, W2) + b2)
L3 = tf.nn.elu(tf.matmul(L2, W3) + b3)
Hypothesis = tf.nn.tanh(tf.matmul(L3, W4) + b4)

cost = tf.reduce_mean(tf.square(Y-Hypothesis))
train = tf.train.AdamOptimizer(learning_rate=learning_rate,beta1=0.5).minimize(cost)

## Train NN & Predict

In [10]:
with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    
    for step in range(100000):
        SGDorder = Sequence[step % 20] # training을 0번부터 20번까지 순차적으로 돌린다.
        session.run(train, feed_dict ={X: X_TrainingData[SGDorder: SGDorder + 1,:], Y: Y_TrainingData[SGDorder: SGDorder + 1]})
        #session.run(train, feed_dict ={X: X_TrainingData, Y: Y_TrainingData})
        
        if step % 1000 == 0:
            print (SGDorder,step, session.run(cost, feed_dict = {X: X_TrainingData, Y: Y_TrainingData}))
            #print (W1)
            #print (b1)
        if step % 20 == 0:
            random.shuffle(Sequence)
            
    Predict_to_TestData = session.run(Hypothesis, feed_dict={X : X_TestData})
    Predict_to_TrainingData = session.run(Hypothesis, feed_dict={X : X_TrainingData})
    Predict_to_EntireData = session.run(Hypothesis, feed_dict = {X : X_NormalizedData})
    
    print("R_square of EntireData")
    print(r2_score(Y_NormalizedData, Predict_to_EntireData))
    print("R_square of TestData")
    print(r2_score(Y_TestData, Predict_to_TestData))
    print("Result of predicting TrainingData")
    print(Predict_to_TrainingData)
    print("Result of predicting TestData")
    print(Predict_to_TestData)
    print("Y_NormalizedData")
    print(Y_NormalizedData)

19 0 0.0596981
4 1000 0.0897008
11 2000 0.0889376
5 3000 0.0938809
7 4000 0.0967513
0 5000 0.104859
18 6000 0.102976
0 7000 0.105254
17 8000 0.0989922
17 9000 0.109437
12 10000 0.110436
1 11000 0.109223
5 12000 0.103502
0 13000 0.107501
13 14000 0.123156
5 15000 0.120605
8 16000 0.120888
3 17000 0.107334
18 18000 0.109463
2 19000 0.110523
16 20000 0.0940826
2 21000 0.117304
13 22000 0.104647
19 23000 0.109144
2 24000 0.112648
13 25000 0.126408
12 26000 0.117465
2 27000 0.121186
6 28000 0.111618
0 29000 0.110644
0 30000 0.115826
8 31000 0.109355
5 32000 0.112198
2 33000 0.116038
5 34000 0.117024
12 35000 0.109387
3 36000 0.120815
0 37000 0.107375
17 38000 0.11584
14 39000 0.112182
17 40000 0.114894
15 41000 0.112776
5 42000 0.114154
10 43000 0.109137
3 44000 0.113969
3 45000 0.114935
6 46000 0.115283
14 47000 0.116135
4 48000 0.120806
5 49000 0.116735
10 50000 0.114097
16 51000 0.116238
7 52000 0.114451
2 53000 0.115835
16 54000 0.118529
11 55000 0.109501
4 56000 0.117243
2 57000 0.1146

# 5th ReLU, tanh, 4, 50, 50, 50, 1
R_square of EntireData
0.642259338803
R_square of TestData
-5.50543815376

## Model

In [11]:
learning_rate = 0.001
tf.set_random_seed(777)

n_InputUnit = 4
n_1stHiddenUnit=50
n_2ndHiddenUnit=50
n_3rdHiddenUnit=50
n_OutputUnit = 1

W1 = tf.Variable(tf.random_normal([n_InputUnit, n_1stHiddenUnit],stddev=0.01))
W2 = tf.Variable(tf.random_normal([n_1stHiddenUnit, n_2ndHiddenUnit],stddev=0.01))
W3 = tf.Variable(tf.random_normal([n_2ndHiddenUnit, n_3rdHiddenUnit],stddev=0.01))
W4 = tf.Variable(tf.random_normal([n_3rdHiddenUnit, n_OutputUnit],stddev=0.01))

b1 = tf.Variable(tf.random_normal([n_1stHiddenUnit], stddev=0.01))
b2 = tf.Variable(tf.random_normal([n_2ndHiddenUnit], stddev=0.01))
b3 = tf.Variable(tf.random_normal([n_3rdHiddenUnit], stddev=0.01))
b4 = tf.Variable(tf.random_normal([n_OutputUnit], stddev=0.01))

X = tf.placeholder(tf.float32, [None, 4]) # tensorflow의 placeholder라는 변수를 선언
Y = tf.placeholder(tf.float32, [None,])

L1 = tf.nn.relu(tf.matmul(X, W1) + b1)
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)
L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)
Hypothesis = tf.nn.tanh(tf.matmul(L3, W4) + b4)

cost = tf.reduce_mean(tf.square(Y-Hypothesis))
train = tf.train.AdamOptimizer(learning_rate=learning_rate,beta1=0.5).minimize(cost)

## Train NN & Predict

In [12]:
with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    
    for step in range(100000):
        SGDorder = Sequence[step % 20] # training을 0번부터 20번까지 순차적으로 돌린다.
        session.run(train, feed_dict ={X: X_TrainingData[SGDorder: SGDorder + 1,:], Y: Y_TrainingData[SGDorder: SGDorder + 1]})
        #session.run(train, feed_dict ={X: X_TrainingData, Y: Y_TrainingData})
        
        if step % 1000 == 0:
            print (SGDorder,step, session.run(cost, feed_dict = {X: X_TrainingData, Y: Y_TrainingData}))
            #print (W1)
            #print (b1)
        if step % 20 == 0:
            random.shuffle(Sequence)
            
    Predict_to_TestData = session.run(Hypothesis, feed_dict={X : X_TestData})
    Predict_to_TrainingData = session.run(Hypothesis, feed_dict={X : X_TrainingData})
    Predict_to_EntireData = session.run(Hypothesis, feed_dict = {X : X_NormalizedData})
    
    print("R_square of EntireData")
    print(r2_score(Y_NormalizedData, Predict_to_EntireData))
    print("R_square of TestData")
    print(r2_score(Y_TestData, Predict_to_TestData))
    print("Result of predicting TrainingData")
    print(Predict_to_TrainingData)
    print("Result of predicting TestData")
    print(Predict_to_TestData)
    print("Y_NormalizedData")
    print(Y_NormalizedData)

14 0 0.0591693
15 1000 0.0975256
10 2000 0.112358
13 3000 0.115108
2 4000 0.113996
6 5000 0.113809
15 6000 0.117102
2 7000 0.115923
6 8000 0.115322
8 9000 0.115124
0 10000 0.11466
6 11000 0.114477
4 12000 0.119248
0 13000 0.120742
9 14000 0.115877
9 15000 0.114252
3 16000 0.115507
4 17000 0.118868
19 18000 0.116578
18 19000 0.119748
5 20000 0.115859
10 21000 0.114764
17 22000 0.112438
11 23000 0.118114
10 24000 0.120242
16 25000 0.109328
5 26000 0.114462
16 27000 0.137628
14 28000 0.113516
14 29000 0.115665
18 30000 0.112131
16 31000 0.11147
8 32000 0.114249
9 33000 0.114376
6 34000 0.115059
19 35000 0.115385
10 36000 0.113245
16 37000 0.113158
8 38000 0.112646
5 39000 0.114132
1 40000 0.11559
8 41000 0.112814
9 42000 0.116471
10 43000 0.115354
1 44000 0.11462
10 45000 0.115058
5 46000 0.11491
11 47000 0.114842
10 48000 0.117516
7 49000 0.113884
5 50000 0.115292
0 51000 0.115428
11 52000 0.116388
19 53000 0.114227
3 54000 0.11495
13 55000 0.113052
4 56000 0.116101
9 57000 0.115329
8 58

# 6th eLU, tanh, 4, 50, 40, 30, 20, 1
R_square of EntireData
0.802634340865
R_square of TestData
-2.54453531064

## Model

In [13]:
learning_rate = 0.001
tf.set_random_seed(777)

n_InputUnit = 4
n_1stHiddenUnit=50
n_2ndHiddenUnit=40
n_3rdHiddenUnit=30
n_4thHiddenUnit=20
n_OutputUnit = 1

W1 = tf.Variable(tf.random_normal([n_InputUnit, n_1stHiddenUnit],stddev=0.01))
W2 = tf.Variable(tf.random_normal([n_1stHiddenUnit, n_2ndHiddenUnit],stddev=0.01))
W3 = tf.Variable(tf.random_normal([n_2ndHiddenUnit, n_3rdHiddenUnit],stddev=0.01))
W4 = tf.Variable(tf.random_normal([n_3rdHiddenUnit, n_4thHiddenUnit],stddev=0.01))
W5 = tf.Variable(tf.random_normal([n_4thHiddenUnit, n_OutputUnit],stddev=0.01))

b1 = tf.Variable(tf.random_normal([n_1stHiddenUnit], stddev=0.01))
b2 = tf.Variable(tf.random_normal([n_2ndHiddenUnit], stddev=0.01))
b3 = tf.Variable(tf.random_normal([n_3rdHiddenUnit], stddev=0.01))
b4 = tf.Variable(tf.random_normal([n_4thHiddenUnit], stddev=0.01))
b5 = tf.Variable(tf.random_normal([n_OutputUnit], stddev=0.01))

X = tf.placeholder(tf.float32, [None, 4]) # tensorflow의 placeholder라는 변수를 선언
Y = tf.placeholder(tf.float32, [None,])

L1 = tf.nn.elu(tf.matmul(X, W1) + b1)
L2 = tf.nn.elu(tf.matmul(L1, W2) + b2)
L3 = tf.nn.elu(tf.matmul(L2, W3) + b3)
L4 = tf.nn.elu(tf.matmul(L3, W4) + b4)
Hypothesis = tf.nn.tanh(tf.matmul(L4, W5) + b5)

cost = tf.reduce_mean(tf.square(Y-Hypothesis))
train = tf.train.AdamOptimizer(learning_rate=learning_rate,beta1=0.5).minimize(cost)

## Train NN & Predict

In [14]:
with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    
    for step in range(100000):
        SGDorder = Sequence[step % 20] # training을 0번부터 20번까지 순차적으로 돌린다.
        session.run(train, feed_dict ={X: X_TrainingData[SGDorder: SGDorder + 1,:], Y: Y_TrainingData[SGDorder: SGDorder + 1]})
        #session.run(train, feed_dict ={X: X_TrainingData, Y: Y_TrainingData})
        
        if step % 1000 == 0:
            print (SGDorder,step, session.run(cost, feed_dict = {X: X_TrainingData, Y: Y_TrainingData}))
            #print (W1)
            #print (b1)
        if step % 20 == 0:
            random.shuffle(Sequence)
            
    Predict_to_TestData = session.run(Hypothesis, feed_dict={X : X_TestData})
    Predict_to_TrainingData = session.run(Hypothesis, feed_dict={X : X_TrainingData})
    Predict_to_EntireData = session.run(Hypothesis, feed_dict = {X : X_NormalizedData})
    
    print("R_square of EntireData")
    print(r2_score(Y_NormalizedData, Predict_to_EntireData))
    print("R_square of TestData")
    print(r2_score(Y_TestData, Predict_to_TestData))
    print("Result of predicting TrainingData")
    print(Predict_to_TrainingData)
    print("Result of predicting TestData")
    print(Predict_to_TestData)
    print("Y_NormalizedData")
    print(Y_NormalizedData)

9 0 0.0593363
8 1000 0.0982946
6 2000 0.107218
11 3000 0.0983916
13 4000 0.0972404
18 5000 0.0921926
9 6000 0.0979304
8 7000 0.102597
6 8000 0.103848
2 9000 0.0978132
19 10000 0.108216
13 11000 0.0959886
0 12000 0.100179
6 13000 0.105069
1 14000 0.109726
5 15000 0.109577
16 16000 0.129259
19 17000 0.118885
7 18000 0.0961095
15 19000 0.116228
2 20000 0.113653
6 21000 0.113323
5 22000 0.111004
6 23000 0.114761
0 24000 0.113706
14 25000 0.109272
7 26000 0.114584
8 27000 0.119428
9 28000 0.111844
8 29000 0.112512
11 30000 0.114559
6 31000 0.121337
6 32000 0.112563
5 33000 0.112516
8 34000 0.113221
8 35000 0.114945
3 36000 0.100093
3 37000 0.114985
13 38000 0.116057
18 39000 0.115069
5 40000 0.113988
11 41000 0.115039
2 42000 0.114887
18 43000 0.115451
18 44000 0.112967
11 45000 0.119302
8 46000 0.115622
12 47000 0.114324
19 48000 0.11638
3 49000 0.108391
16 50000 0.121481
0 51000 0.129851
19 52000 0.104721
0 53000 0.112609
8 54000 0.116833
2 55000 0.115296
10 56000 0.114775
16 57000 0.1170

# 7th eLU, tanh, 4, 100, 100, 50, 1
R_square of EntireData
0.702912816686
R_square of TestData
-4.37658055001

## Model

In [15]:
learning_rate = 0.001
tf.set_random_seed(777)

n_InputUnit = 4
n_1stHiddenUnit=100
n_2ndHiddenUnit=100
n_3rdHiddenUnit=50
n_OutputUnit = 1

W1 = tf.Variable(tf.random_normal([n_InputUnit, n_1stHiddenUnit],stddev=0.01))
W2 = tf.Variable(tf.random_normal([n_1stHiddenUnit, n_2ndHiddenUnit],stddev=0.01))
W3 = tf.Variable(tf.random_normal([n_2ndHiddenUnit, n_3rdHiddenUnit],stddev=0.01))
W4 = tf.Variable(tf.random_normal([n_3rdHiddenUnit, n_OutputUnit],stddev=0.01))

b1 = tf.Variable(tf.random_normal([n_1stHiddenUnit], stddev=0.01))
b2 = tf.Variable(tf.random_normal([n_2ndHiddenUnit], stddev=0.01))
b3 = tf.Variable(tf.random_normal([n_3rdHiddenUnit], stddev=0.01))
b4 = tf.Variable(tf.random_normal([n_OutputUnit], stddev=0.01))

X = tf.placeholder(tf.float32, [None, 4]) # tensorflow의 placeholder라는 변수를 선언
Y = tf.placeholder(tf.float32, [None,])

L1 = tf.nn.elu(tf.matmul(X, W1) + b1)
L2 = tf.nn.elu(tf.matmul(L1, W2) + b2)
L3 = tf.nn.elu(tf.matmul(L2, W3) + b3)
Hypothesis = tf.nn.tanh(tf.matmul(L3, W4) + b4)

cost = tf.reduce_mean(tf.square(Y-Hypothesis))
train = tf.train.AdamOptimizer(learning_rate=learning_rate,beta1=0.5).minimize(cost)

## Train NN & Predict

In [16]:
with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    
    for step in range(100000):
        SGDorder = Sequence[step % 20] # training을 0번부터 20번까지 순차적으로 돌린다.
        session.run(train, feed_dict ={X: X_TrainingData[SGDorder: SGDorder + 1,:], Y: Y_TrainingData[SGDorder: SGDorder + 1]})
        #session.run(train, feed_dict ={X: X_TrainingData, Y: Y_TrainingData})
        
        if step % 1000 == 0:
            print (SGDorder,step, session.run(cost, feed_dict = {X: X_TrainingData, Y: Y_TrainingData}))
            #print (W1)
            #print (b1)
        if step % 20 == 0:
            random.shuffle(Sequence)
            
    Predict_to_TestData = session.run(Hypothesis, feed_dict={X : X_TestData})
    Predict_to_TrainingData = session.run(Hypothesis, feed_dict={X : X_TrainingData})
    Predict_to_EntireData = session.run(Hypothesis, feed_dict = {X : X_NormalizedData})
    
    print("R_square of EntireData")
    print(r2_score(Y_NormalizedData, Predict_to_EntireData))
    print("R_square of TestData")
    print(r2_score(Y_TestData, Predict_to_TestData))
    print("Result of predicting TrainingData")
    print(Predict_to_TrainingData)
    print("Result of predicting TestData")
    print(Predict_to_TestData)
    print("Y_NormalizedData")
    print(Y_NormalizedData)

4 0 0.0583551
15 1000 0.0863091
2 2000 0.0954702
16 3000 0.0908538
5 4000 0.0976067
5 5000 0.0980117
5 6000 0.103242
14 7000 0.116341
4 8000 0.0999195
10 9000 0.0936804
6 10000 0.113702
6 11000 0.112308
12 12000 0.104366
16 13000 0.119051
10 14000 0.122052
5 15000 0.110311
14 16000 0.119476
2 17000 0.111079
6 18000 0.102843
4 19000 0.133548
0 20000 0.110261
13 21000 0.108627
7 22000 0.116181
17 23000 0.113327
5 24000 0.115769
4 25000 0.117506
1 26000 0.110776
19 27000 0.112315
4 28000 0.113479
19 29000 0.111805
15 30000 0.106639
0 31000 0.116104
17 32000 0.109839
9 33000 0.112591
18 34000 0.116151
1 35000 0.118806
19 36000 0.117676
11 37000 0.106575
8 38000 0.114518
18 39000 0.110236
4 40000 0.123988
0 41000 0.117035
12 42000 0.11829
5 43000 0.115154
11 44000 0.116687
14 45000 0.108413
19 46000 0.112382
10 47000 0.108848
13 48000 0.115522
7 49000 0.113196
8 50000 0.110597
3 51000 0.114747
9 52000 0.117333
12 53000 0.113497
13 54000 0.115426
8 55000 0.113211
0 56000 0.115726
12 57000 0.

# 8th eLU, tanh, 4, 10, 10, 10, 10, 10, 10, 1
R_square of EntireData
0.612517079802
R_square of TestData
-6.02644501071

## Model

In [17]:
learning_rate = 0.001
tf.set_random_seed(777)

n_InputUnit = 4
n_1stHiddenUnit=10
n_2ndHiddenUnit=10
n_3rdHiddenUnit=10
n_4thHiddenUnit=10
n_5thHiddenUnit=10
n_6thHiddenUnit=10
n_OutputUnit = 1

W1 = tf.Variable(tf.random_normal([n_InputUnit, n_1stHiddenUnit],stddev=0.01))
W2 = tf.Variable(tf.random_normal([n_1stHiddenUnit, n_2ndHiddenUnit],stddev=0.01))
W3 = tf.Variable(tf.random_normal([n_2ndHiddenUnit, n_3rdHiddenUnit],stddev=0.01))
W4 = tf.Variable(tf.random_normal([n_3rdHiddenUnit, n_4thHiddenUnit],stddev=0.01))
W5 = tf.Variable(tf.random_normal([n_4thHiddenUnit, n_5thHiddenUnit],stddev=0.01))
W6 = tf.Variable(tf.random_normal([n_5thHiddenUnit, n_6thHiddenUnit],stddev=0.01))
W7 = tf.Variable(tf.random_normal([n_6thHiddenUnit, n_OutputUnit],stddev=0.01))

b1 = tf.Variable(tf.random_normal([n_1stHiddenUnit], stddev=0.01))
b2 = tf.Variable(tf.random_normal([n_2ndHiddenUnit], stddev=0.01))
b3 = tf.Variable(tf.random_normal([n_3rdHiddenUnit], stddev=0.01))
b4 = tf.Variable(tf.random_normal([n_4thHiddenUnit], stddev=0.01))
b5 = tf.Variable(tf.random_normal([n_5thHiddenUnit], stddev=0.01))
b6 = tf.Variable(tf.random_normal([n_6thHiddenUnit], stddev=0.01))
b7 = tf.Variable(tf.random_normal([n_OutputUnit], stddev=0.01))

X = tf.placeholder(tf.float32, [None, 4]) # tensorflow의 placeholder라는 변수를 선언
Y = tf.placeholder(tf.float32, [None,])

L1 = tf.nn.elu(tf.matmul(X, W1) + b1)
L2 = tf.nn.elu(tf.matmul(L1, W2) + b2)
L3 = tf.nn.elu(tf.matmul(L2, W3) + b3)
L4 = tf.nn.elu(tf.matmul(L3, W4) + b4)
L5 = tf.nn.elu(tf.matmul(L4, W5) + b5)
L6 = tf.nn.elu(tf.matmul(L5, W6) + b6)
Hypothesis = tf.nn.tanh(tf.matmul(L6, W7) + b7)

cost = tf.reduce_mean(tf.square(Y-Hypothesis))
train = tf.train.AdamOptimizer(learning_rate=learning_rate,beta1=0.5).minimize(cost)

## Train NN & Predict

In [18]:
with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    
    for step in range(100000):
        SGDorder = Sequence[step % 20] # training을 0번부터 20번까지 순차적으로 돌린다.
        session.run(train, feed_dict ={X: X_TrainingData[SGDorder: SGDorder + 1,:], Y: Y_TrainingData[SGDorder: SGDorder + 1]})
        #session.run(train, feed_dict ={X: X_TrainingData, Y: Y_TrainingData})
        
        if step % 1000 == 0:
            print (SGDorder,step, session.run(cost, feed_dict = {X: X_TrainingData, Y: Y_TrainingData}))
            #print (W1)
            #print (b1)
        if step % 20 == 0:
            random.shuffle(Sequence)
            
    Predict_to_TestData = session.run(Hypothesis, feed_dict={X : X_TestData})
    Predict_to_TrainingData = session.run(Hypothesis, feed_dict={X : X_TrainingData})
    Predict_to_EntireData = session.run(Hypothesis, feed_dict = {X : X_NormalizedData})
    
    print("R_square of EntireData")
    print(r2_score(Y_NormalizedData, Predict_to_EntireData))
    print("R_square of TestData")
    print(r2_score(Y_TestData, Predict_to_TestData))
    print("Result of predicting TrainingData")
    print(Predict_to_TrainingData)
    print("Result of predicting TestData")
    print(Predict_to_TestData)
    print("Y_NormalizedData")
    print(Y_NormalizedData)

16 0 0.0582934
16 1000 0.05751
8 2000 0.0575095
4 3000 0.0966541
15 4000 0.0964972
11 5000 0.0968752
2 6000 0.0926198
10 7000 0.0944618
10 8000 0.0906815
16 9000 0.100562
14 10000 0.103533
12 11000 0.101885
5 12000 0.10769
5 13000 0.102072
17 14000 0.105102
13 15000 0.109704
9 16000 0.107381
8 17000 0.106167
14 18000 0.107517
5 19000 0.101307
1 20000 0.108314
17 21000 0.112364
6 22000 0.114066
9 23000 0.105962
17 24000 0.118172
17 25000 0.120901
3 26000 0.105802
16 27000 0.0977308
19 28000 0.106979
11 29000 0.11615
7 30000 0.115795
14 31000 0.131419
15 32000 0.119447
15 33000 0.111924
14 34000 0.121741
9 35000 0.112085
10 36000 0.110781
3 37000 0.11256
4 38000 0.114682
11 39000 0.117384
15 40000 0.123782
19 41000 0.117423
17 42000 0.114012
11 43000 0.115229
9 44000 0.108603
6 45000 0.124413
14 46000 0.12082
6 47000 0.124854
7 48000 0.100911
19 49000 0.111381
7 50000 0.113548
3 51000 0.117894
9 52000 0.114129
2 53000 0.116448
0 54000 0.116112
0 55000 0.112226
14 56000 0.110771
16 57000 

# 9th eLU, tanh, 4, 500, 500, 100, 1 
R_square of EntireData
-18.8171709825
R_square of TestData
-51.8248455069

## Model

In [19]:
learning_rate = 0.001
tf.set_random_seed(777)

n_InputUnit = 4
n_1stHiddenUnit=500
n_2ndHiddenUnit=500
n_3rdHiddenUnit=100
n_OutputUnit = 1

W1 = tf.Variable(tf.random_normal([n_InputUnit, n_1stHiddenUnit],stddev=0.01))
W2 = tf.Variable(tf.random_normal([n_1stHiddenUnit, n_2ndHiddenUnit],stddev=0.01))
W3 = tf.Variable(tf.random_normal([n_2ndHiddenUnit, n_3rdHiddenUnit],stddev=0.01))
W4 = tf.Variable(tf.random_normal([n_3rdHiddenUnit, n_OutputUnit],stddev=0.01))

b1 = tf.Variable(tf.random_normal([n_1stHiddenUnit], stddev=0.01))
b2 = tf.Variable(tf.random_normal([n_2ndHiddenUnit], stddev=0.01))
b3 = tf.Variable(tf.random_normal([n_3rdHiddenUnit], stddev=0.01))
b4 = tf.Variable(tf.random_normal([n_OutputUnit], stddev=0.01))

X = tf.placeholder(tf.float32, [None, 4]) # tensorflow의 placeholder라는 변수를 선언
Y = tf.placeholder(tf.float32, [None,])

L1 = tf.nn.elu(tf.matmul(X, W1) + b1)
L2 = tf.nn.elu(tf.matmul(L1, W2) + b2)
L3 = tf.nn.elu(tf.matmul(L2, W3) + b3)
Hypothesis = tf.nn.tanh(tf.matmul(L3, W4) + b4)

cost = tf.reduce_mean(tf.square(Y-Hypothesis))
train = tf.train.AdamOptimizer(learning_rate=learning_rate,beta1=0.5).minimize(cost)

## Train NN & Predict

In [20]:
with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    
    for step in range(100000):
        SGDorder = Sequence[step % 20] # training을 0번부터 20번까지 순차적으로 돌린다.
        session.run(train, feed_dict ={X: X_TrainingData[SGDorder: SGDorder + 1,:], Y: Y_TrainingData[SGDorder: SGDorder + 1]})
        #session.run(train, feed_dict ={X: X_TrainingData, Y: Y_TrainingData})
        
        if step % 1000 == 0:
            print (SGDorder,step, session.run(cost, feed_dict = {X: X_TrainingData, Y: Y_TrainingData}))
            #print (W1)
            #print (b1)
        if step % 20 == 0:
            random.shuffle(Sequence)
            
    Predict_to_TestData = session.run(Hypothesis, feed_dict={X : X_TestData})
    Predict_to_TrainingData = session.run(Hypothesis, feed_dict={X : X_TrainingData})
    Predict_to_EntireData = session.run(Hypothesis, feed_dict = {X : X_NormalizedData})
    
    print("R_square of EntireData")
    print(r2_score(Y_NormalizedData, Predict_to_EntireData))
    print("R_square of TestData")
    print(r2_score(Y_TestData, Predict_to_TestData))
    print("Result of predicting TrainingData")
    print(Predict_to_TrainingData)
    print("Result of predicting TestData")
    print(Predict_to_TestData)
    print("Y_NormalizedData")
    print(Y_NormalizedData)

0 0 0.0578898
16 1000 0.0841027
16 2000 0.0974794
7 3000 0.100917
13 4000 0.11413
1 5000 0.115132
10 6000 0.108221
10 7000 0.119891
4 8000 0.125731
18 9000 0.10196
17 10000 0.109426
3 11000 0.107223
9 12000 0.115586
4 13000 0.147572
1 14000 0.10946
4 15000 0.119975
5 16000 0.124041
2 17000 0.116147
3 18000 0.113332
13 19000 0.119378
19 20000 0.118228
7 21000 0.113308
15 22000 0.113621
17 23000 0.109031
9 24000 0.117394
12 25000 0.116164
6 26000 0.105244
16 27000 0.121559
4 28000 0.110353
16 29000 0.115952
8 30000 0.113153
7 31000 0.112761
8 32000 0.11676
14 33000 0.10687
9 34000 0.114125
7 35000 0.112777
13 36000 0.116349
12 37000 0.113663
6 38000 0.112706
3 39000 0.122141
3 40000 0.111878
7 41000 0.116525
10 42000 0.111899
1 43000 0.116429
15 44000 0.116687
16 45000 0.12634
14 46000 0.11328
14 47000 0.114212
15 48000 0.116626
2 49000 0.118573
10 50000 0.134119
6 51000 0.120963
2 52000 0.115302
1 53000 0.114458
14 54000 0.115873
18 55000 0.11608
16 56000 0.10988
6 57000 0.115464
19 580

# 10th eLU, tanh, 4, 100, 100, 100, 50, 1 
R_square of EntireData
0.754092927381
R_square of TestData
-3.46744631107

## Model

In [21]:
learning_rate = 0.001
tf.set_random_seed(777)

n_InputUnit = 4
n_1stHiddenUnit=100
n_2ndHiddenUnit=100
n_3rdHiddenUnit=100
n_4thHiddenUnit=50
n_OutputUnit = 1

W1 = tf.Variable(tf.random_normal([n_InputUnit, n_1stHiddenUnit],stddev=0.01))
W2 = tf.Variable(tf.random_normal([n_1stHiddenUnit, n_2ndHiddenUnit],stddev=0.01))
W3 = tf.Variable(tf.random_normal([n_2ndHiddenUnit, n_3rdHiddenUnit],stddev=0.01))
W4 = tf.Variable(tf.random_normal([n_3rdHiddenUnit, n_4thHiddenUnit],stddev=0.01))
W5 = tf.Variable(tf.random_normal([n_4thHiddenUnit, n_OutputUnit],stddev=0.01))

b1 = tf.Variable(tf.random_normal([n_1stHiddenUnit], stddev=0.01))
b2 = tf.Variable(tf.random_normal([n_2ndHiddenUnit], stddev=0.01))
b3 = tf.Variable(tf.random_normal([n_3rdHiddenUnit], stddev=0.01))
b4 = tf.Variable(tf.random_normal([n_4thHiddenUnit], stddev=0.01))
b5 = tf.Variable(tf.random_normal([n_OutputUnit], stddev=0.01))

X = tf.placeholder(tf.float32, [None, 4]) # tensorflow의 placeholder라는 변수를 선언
Y = tf.placeholder(tf.float32, [None,])

L1 = tf.nn.elu(tf.matmul(X, W1) + b1)
L2 = tf.nn.elu(tf.matmul(L1, W2) + b2)
L3 = tf.nn.elu(tf.matmul(L2, W3) + b3)
L4 = tf.nn.elu(tf.matmul(L3, W4) + b4)
Hypothesis = tf.nn.tanh(tf.matmul(L4, W5) + b5)

cost = tf.reduce_mean(tf.square(Y-Hypothesis))
train = tf.train.AdamOptimizer(learning_rate=learning_rate,beta1=0.5).minimize(cost)

## Train NN & Predict

In [22]:
with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    
    for step in range(100000):
        SGDorder = Sequence[step % 20] # training을 0번부터 20번까지 순차적으로 돌린다.
        session.run(train, feed_dict ={X: X_TrainingData[SGDorder: SGDorder + 1,:], Y: Y_TrainingData[SGDorder: SGDorder + 1]})
        #session.run(train, feed_dict ={X: X_TrainingData, Y: Y_TrainingData})
        
        if step % 1000 == 0:
            print (SGDorder,step, session.run(cost, feed_dict = {X: X_TrainingData, Y: Y_TrainingData}))
            #print (W1)
            #print (b1)
        if step % 20 == 0:
            random.shuffle(Sequence)
            
    Predict_to_TestData = session.run(Hypothesis, feed_dict={X : X_TestData})
    Predict_to_TrainingData = session.run(Hypothesis, feed_dict={X : X_TrainingData})
    Predict_to_EntireData = session.run(Hypothesis, feed_dict = {X : X_NormalizedData})
    
    print("R_square of EntireData")
    print(r2_score(Y_NormalizedData, Predict_to_EntireData))
    print("R_square of TestData")
    print(r2_score(Y_TestData, Predict_to_TestData))
    print("Result of predicting TrainingData")
    print(Predict_to_TrainingData)
    print("Result of predicting TestData")
    print(Predict_to_TestData)
    print("Y_NormalizedData")
    print(Y_NormalizedData)

5 0 0.0590917
5 1000 0.118709
14 2000 0.0932212
0 3000 0.0979851
0 4000 0.106285
2 5000 0.105408
6 6000 0.105537
7 7000 0.113743
7 8000 0.118868
8 9000 0.100202
4 10000 0.114723
8 11000 0.112881
1 12000 0.114681
2 13000 0.104768
5 14000 0.120103
16 15000 0.114904
9 16000 0.113971
16 17000 0.111085
2 18000 0.115722
5 19000 0.113407
17 20000 0.114827
13 21000 0.115482
3 22000 0.103965
11 23000 0.117315
11 24000 0.113727
11 25000 0.115044
0 26000 0.117305
12 27000 0.115627
10 28000 0.114337
18 29000 0.116013
8 30000 0.11508
1 31000 0.114487
16 32000 0.114685
19 33000 0.114983
2 34000 0.116197
6 35000 0.114229
1 36000 0.114139
4 37000 0.111973
15 38000 0.113882
18 39000 0.116054
10 40000 0.121342
13 41000 0.112518
19 42000 0.115446
5 43000 0.110768
19 44000 0.114641
5 45000 0.114582
0 46000 0.114922
2 47000 0.116096
10 48000 0.122311
1 49000 0.116251
15 50000 0.115628
6 51000 0.115042
2 52000 0.114676
7 53000 0.114749
11 54000 0.115082
6 55000 0.115064
14 56000 0.118031
1 57000 0.115032
15